In [2]:
class Node:
    def __init__(self, ID, x, y, demand):
        self.ID = ID
        self.x = x
        self.y = y
        self.demand = demand
        self.inRoute = None
        self.isInterior = False
        self.dnEdge = None
        self.ndEdge = None
        self.isLinkedToStart = False
        self.isLinkedToFinish = False

class Edge:
    def __init__(self, origin, end):
        self.origin = origin
        self.end = end
        self.cost = 0.0
        self.savings = 0.0
        self.invEdge = None
        self.efficiency = 0.0

class Route:
    def __init__(self):
        self.cost = 0.0
        self.edges = []
        self.demand = 0.0

    def reverse(self):
        size = len(self.edges)
        for i in range(size):
            edge = self.edges[i]
            invEdge = edge.invEdge
            self.edges.remove(edge)
            self.edges.insert(0, invEdge)

class Solution:
    last_ID = -1

    def __init__(self):
        Solution.last_ID += 1
        self.ID = Solution.last_ID
        self.routes = []
        self.cost = 0.0
        self.demand = 0.0

In [3]:
import math
import operator
import os

vehCap = 100.0  # Update vehicle capacity for each instance
instanceName = 'A-n80-k10'  # instance to be searched in drop box

# Correct file path with os.path.join
fileName = os.path.join(r'C:\Users\shiri\Desktop\Hot Topics\data', instanceName + '_input_nodes.txt')

# Verify if the file exists before proceeding
if os.path.exists(fileName):
    # Search through the instances with filename
    with open(fileName) as instance:
        i = 0
        nodes = []
        for line in instance:
            # Split the line into parts and convert to float
            data = [float(x) for x in line.split()]
            # Ensure there are exactly 3 values (x, y, demand)
            if len(data) == 3:
                aNode = Node(i, data[0], data[1], data[2])
                nodes.append(aNode)
                i += 1
else:
    print(f"File not found: {fileName}")


In [4]:
""" Construct edges with costs and savings list from nodes """
depot = nodes [0] # node is the depot

for node in nodes [1:]: # excludes the depot
    dnEdge = Edge(depot, node) # creates the (depot, node) edge (arc) 
    ndEdge = Edge(node, depot)
    dnEdge.invEdge = ndEdge # sets the inverse edge (arc)
    ndEdge.invEdge = dnEdge
    #compute the Euclidean distance as cost
    dnEdge.cost = math.sqrt((node.x - depot.x)**2 + (node.y - depot.y)**2)
    ndEdge.cost = dnEdge.cost # assume symmetric costs
    # save in node a reference to the (depot, node) edge (arc)
    node.dnEdge = dnEdge
    node.ndEdge = ndEdge

savingsList = []
for i in range(1, len(nodes) - 1): # excludes the depot 
    iNode = nodes[i]
    for j in range( i + 1, len (nodes)):
        jNode = nodes[j]
        ijEdge = Edge (iNode, jNode) # creates the (i, j)
        jiEdge = Edge (jNode, iNode)
        ijEdge.invEdge = jiEdge # sets the inverse edge (arc)
        jiEdge.invEdge = ijEdge
        # compute the Euclidean distance as cost
        ijEdge.cost = math.sqrt((jNode.x - iNode.x)**2 + (jNode.y - iNode.y) **2) 
        jiEdge.cost = ijEdge.cost # assume symmetric costs
        #compute savings as proposed by Clark & Wright
        ijEdge.savings = iNode.ndEdge.cost + jNode.dnEdge.cost - ijEdge.cost 
        jiEdge.savings = ijEdge.savings
        # save one edge in the savings list 
        savingsList.append(ijEdge)
#sort the list of edges from higher to lower savings
savingsList.sort(key = operator.attrgetter("savings"), reverse = True)

In [5]:

'''Construct the dummy solution'''
sol = Solution()

for node in nodes [1:]: # excludes the depot
    dnEdge = node.dnEdge # get the (depot, node) edge
    ndEdge = node.ndEdge
    dndRoute = Route() # construct the route (depot, node, depot)
    dndRoute.edges.append(dnEdge)
    dndRoute.demand += node.demand 
    dndRoute.cost += dnEdge.cost
    dndRoute.edges.append(ndEdge)
    dndRoute.cost += ndEdge.cost
    node.inRoute = dndRoute # save in node a reference to its current route
    node.isInterior = False # this node is currently exterior (connected to depot) 
    sol.routes.append(dndRoute) # add this route to the solution
    sol.cost += dndRoute.cost
    sol.demand += dndRoute.demand
'''
Perform the edge-selection & routing-merging iterative process'''
def checkMergingConditions(iNode, jNode, iRoute, jRoute):
    # condition 1: iRoute and jRoure are not the same route object 
    if iRoute == jRoute: return False
    # condition 2: both nodes are exterior nodes in their respective routes 
    if iNode.isInterior == True or jNode.isInterior == True: return False 
    # condition 3: demand after merging can be covered by a single vehicle 
    if vehCap< iRoute.demand + jRoute.demand: return False
    # else, merging is feasible
    return True

def getDepotEdge (aRoute, aNode):
    '''returns the edge in aRoute that contains aNode and the depot (it will be the first or the last one) '''
    # check if first edge in aRoute contains aNode and depot
    origin = aRoute.edges[0].origin
    end = aRoute.edges[0].end
    if ((origin == aNode and end == depot) or (origin == depot and end == aNode)):
        return aRoute.edges[0]
    else: # return last edge in aRoute
        return aRoute.edges[-1]

In [6]:
while len(savingsList) > 0: # list is not empty
    ijEdge = savingsList.pop(0) # select the next edge from the list
    #determine the nodes i<j that define the edge 
    iNode = ijEdge.origin
    jNode = ijEdge.end
    # determine the routes associated with each node
    iRoute = iNode.inRoute
    jRoute = jNode.inRoute
    # check if merge is possible 
    isMergeFeasible = checkMergingConditions (iNode, jNode, iRoute, jRoute)
    # if all necessary conditions are satisfied, merge 
    if isMergeFeasible == True:
        # iRoute will contain either edge (depot, i) or edge (i, depot) 
        iEdge = getDepotEdge(iRoute, iNode) # iEdge is either (0,i) or (1,0)
        # remove iEdge from iRoute and update iRoute cost
        iRoute.edges.remove(iEdge)
        iRoute.cost -= iEdge.cost
        # if there are multiple edges in iRoute, then i will be interior 
        if len(iRoute.edges) > 1: iNode.isInterior = True
        # if new iRoute does not start at 0 it must be reversed
        if iRoute.edges[0].origin != depot: iRoute.reverse()
        #jRoute will contain either edge (depot, j) or edge (j, depot)
        jEdge = getDepotEdge (jRoute, jNode) # jEdge is either (0,j) or (j,0)
        # remove jEdge from jRoute and update jRoute cost
        jRoute.edges.remove(jEdge)
        jRoute.cost -= jEdge.cost
        #if there are multiple edges in jRute, then j will be interior 
        if len(jRoute.edges) > 1: jNode.isInterior = True
        # if new jRoute starts at 0 it must be reversed
        if jRoute.edges[0].origin == depot: jRoute.reverse() 
        #add ijEdge to iRoute
        iRoute.edges.append(ijEdge)
        iRoute.cost += ijEdge.cost
        iRoute.demand += jNode.demand
        jNode.inRoute = iRoute
        #add jRoute to new iRoute 
        for edge in jRoute.edges:
            iRoute.edges.append(edge)
            iRoute.cost += edge.cost 
            iRoute.demand += edge.end.demand 
            edge.end.inRoute = iRoute
        # delete jRoute from emerging solution
        sol.cost -= ijEdge.savings
        sol.routes.remove(jRoute)

In [7]:
print('Cost of C&W savings sol =', ":.{}f".format(sol.cost, 2))
# Iterate through the routes in the solution
for route in sol.routes:
    s = str(0)    
    # Iterate through the edges in the route
    for edge in route.edges:
        s = s + '-' + str(edge.end.ID)
    
    # Print the route and its cost
    print('Route: ' + s + ' || cost = ' + "{:.{}f}".format(route.cost, 2))


Cost of C&W savings sol = :.1860.9424980238105f
Route: 0-1-63-10-7-21-0 || cost = 130.70
Route: 0-64-17-31-27-59-5-23-62-0 || cost = 188.89
Route: 0-24-6-30-78-8-37-2-34-71-0 || cost = 220.95
Route: 0-11-52-28-79-18-48-14-0 || cost = 193.22
Route: 0-13-74-39-60-29-44-12-0 || cost = 143.38
Route: 0-46-20-75-25-41-15-55-9-54-72-0 || cost = 248.96
Route: 0-33-47-56-69-65-35-26-19-57-61-16-43-68-0 || cost = 307.36
Route: 0-36-38-66-67-53-3-77-51-0 || cost = 145.84
Route: 0-40-42-73-49-0 || cost = 87.05
Route: 0-70-76-50-45-22-4-32-58-0 || cost = 194.59
